In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px

N, K, lags = 10, 25, True
directory_path = r"C:\Users\simeo\Documents"
output_path = os.path.join(directory_path, f'final_features_N{N}_K{K}{"_Corr" if lags else ""}.csv')
all_features_df = pd.read_csv(output_path, index_col=0)

In [ ]:
# Combine the reordered columns with the rest of the variables
ordered_columns = (
    [col for col in all_features_df.columns if 'proportion' in col] +
    [col for col in all_features_df.columns if 'mean' in col] +
    [col for col in all_features_df.columns if 'variance' in col] +
    [col for col in all_features_df.columns if 'skew' in col] +
    [col for col in all_features_df.columns if 'Frequency' in col] +
    [col for col in all_features_df.columns if 'Magnitude' in col] +
    [col for col in all_features_df.columns if '_x_' in col]
)

# Reorder the proportion variables to start from bin 0 for every protein, then bin 1, and so on
proportion_columns = sorted(
    [col for col in ordered_columns if 'proportion' in col],
    key=lambda x: (x.split('_')[0], int(x.split('_bin_')[1].split('_')[0]))
)

# Reorder the mean variables to start from bin 0 for every protein, then bin 1, and so on
mean_columns = sorted(
    [col for col in ordered_columns if 'mean' in col],
    key=lambda x: (x.split('_')[0], int(x.split('_bin_')[1].split('_')[0]))
)

# Reorder the variance variables to start from bin 0 for every protein, then bin 1, and so on
variance_columns = sorted(
    [col for col in ordered_columns if 'variance' in col],
    key=lambda x: (x.split('_')[0], int(x.split('_bin_')[1].split('_')[0]))
)

# Reorder the skew variables to start from bin 0 for every protein, then bin 1, and so on
skew_columns = sorted(
    [col for col in ordered_columns if 'skew' in col],
    key=lambda x: (x.split('_')[0], int(x.split('_bin_')[1].split('_')[0]))
)

# Reorder the frequency variables to start with freq 0 for all proteins, then freq 1, and so on
frequency_columns = sorted(
    [col for col in ordered_columns if 'Frequency' in col],
    key=lambda x: int(x.split('_')[-1])
)

# Reorder the magnitude variables to start with mag 0 for all proteins, then mag 1, and so on
magnitude_columns = sorted(
    [col for col in ordered_columns if 'Magnitude' in col],
    key=lambda x: int(x.split('_')[-1])
)

In [ ]:
# Define a function to reorder columns within each group
def reorder_columns(columns, group_type):
    if group_type in ['Proportion', 'Mean', 'Variance', 'Skew']:
        return sorted(
            columns,
            key=lambda x: (int(x.split('_bin_')[1].split('_')[0]), x.split('_')[0])
        )
    elif group_type in ['Frequency', 'Magnitude']:
        return sorted(
            columns,
            key=lambda x: (int(x.split('_')[-1]), x.split('_')[0])
        )
    else:
        return columns

# Define the groups dictionary
groups = {
    'proportion': proportion_columns,
    'mean': mean_columns,
    'variance': variance_columns,
    'skew': skew_columns,
    'Frequency': frequency_columns,
    'Magnitude': magnitude_columns
}

# Combine groups for PCA analysis with reordered columns
combined_groups = {
    'Proportion_Mean_Variance_Skew': (
        reorder_columns(proportion_columns, 'Proportion') +
        reorder_columns(mean_columns, 'Mean') +
        reorder_columns(variance_columns, 'Variance') +
        reorder_columns(skew_columns, 'Skew')
    ),
    'Frequency_Magnitude': (
        reorder_columns(frequency_columns, 'Frequency') +
        reorder_columns(magnitude_columns, 'Magnitude')
    ),
    'Correlations': [col for col in ordered_columns if '_x_' in col]
}

In [ ]:
# Loop through each combined group and generate the plots
for combined_group_name, combined_columns in combined_groups.items():
    # Initialize StandardScaler and PCA
    scaler = StandardScaler()
    pca = PCA()

    # Standardize the data for the current combined group
    standardized_data = scaler.fit_transform(all_features_df[combined_columns].fillna(0))

    # Perform PCA
    pca_transformed_data = pca.fit_transform(standardized_data)

    # Scree plot
    total_variance_explained = pca.explained_variance_ratio_[:4].sum() * 100
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, 5), pca.explained_variance_ratio_[:4], alpha=0.7, color='blue')
    plt.title(f'Scree Plot for {combined_group_name}')
    plt.xlabel('Principal Component')
    plt.ylabel('Variance Explained')
    plt.xticks(range(1, 5))
    plt.text(2.5, max(pca.explained_variance_ratio_[:4]) * 0.9, 
             f'Total Variance Explained: {total_variance_explained:.2f}%', 
             fontsize=12, color='black', ha='center')
    plt.show()

    # PC loadings plot
    pca_components_df = pd.DataFrame(
        pca.components_[:4], 
        columns=combined_columns, 
        index=[f"PC{i+1}" for i in range(4)]
    )
    plt.figure(figsize=(12, 8))
    sns.heatmap(pca_components_df.T, cmap='coolwarm', annot=False, cbar=True)
    plt.title(f'PC Loadings for {combined_group_name}')
    plt.xlabel('Principal Components')
    plt.ylabel('Features')
    plt.show()

In [ ]:
scaler = StandardScaler()
pca = PCA()
standardized_data = scaler.fit_transform(all_features_df.fillna(0))
pca_transformed_data = pca.fit_transform(standardized_data)

# Extract the explained variance ratio for the first 4 PCs
explained_variance_ratio = pca.explained_variance_ratio_[:4]
sum_first_4_pcs = explained_variance_ratio.sum()

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, color='skyblue')
plt.title('PCA Explained Variance Ratio (First 4 PCs)')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.text(2.5, max(pca.explained_variance_ratio_[:4]) * 0.9, 
         f'Total Variance Explained: {sum_first_4_pcs.sum():.2f}%', 
         fontsize=12, color='black', ha='center')
plt.grid()
plt.show()


In [ ]:
# Create a DataFrame for PCA components
pca_components_df = pd.DataFrame(
    pca.components_[:4],  # Use the first 4 principal components
    columns=all_features_df.columns,
    index=[f"PC{i+1}" for i in range(4)]
)

# Use the ordering from the combined groups
ordered_columns = (
    combined_groups['Proportion_Mean_Variance_Skew'] +
    combined_groups['Frequency_Magnitude'] +
    combined_groups['Correlations']
)

# Reorder the columns of the PCA components DataFrame based on ordered_columns
pca_components_ordered_df = pca_components_df[ordered_columns]

# Create a 2x2 figure for PC loadings plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Define colors for the PCs
colors = ['red', 'blue', 'green', 'purple']

# Plot each PC loadings in a separate subplot
for i, ax in enumerate(axes.flatten()):
    pc = pca_components_ordered_df.index[i]
    ax.bar(
        range(len(ordered_columns)),
        pca_components_ordered_df.loc[pc],
        color=colors[i],
        alpha=0.6
    )
    ax.set_title(f'PCA Loadings for {pc}')
    ax.set_xlabel('Features')
    ax.set_ylabel('Loading Value')

    # Define tick positions and labels for key variable groups
    tick_positions = []
    tick_labels = []
    for keyword in ['proportion', 'mean', 'variance', 'skew', 'Frequency', 'Magnitude', '_x_']:
        indices = [j for j, col in enumerate(ordered_columns) if keyword in col]
        if indices:
            # Add start and end ticks
            tick_positions.extend([indices[0], indices[-1]])
            tick_labels.extend([keyword, ''])  # Keyword for middle, empty label for end

    # Set custom ticks
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels, rotation=90)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
scaler = StandardScaler()
pca = PCA()
standardized_data = scaler.fit_transform(all_features_df.fillna(0))
pca_transformed_data = pca.fit_transform(standardized_data)

lncRNA_df_pca = pd.DataFrame(pca_transformed_data, columns=[f"PC{i+1}" for i in range(pca_transformed_data.shape[1])])
lncRNA_df_pca.index = all_features_df.index

Cis_Repressive = ["ENST00000626439.2", "ENST00000424094.6", "ENST00000501176.7",
"ENST00000422420.3", "ENST00000597346.1", "ENST00000447911.6", "ENST00000429829.6"
"ENSMUST00000159731.1", "ENSMUST00000127786.3", "ENST00000414790.10", "ENSMUST00000136359.7"]

Cis_Activating  = ["ENST00000417473.7", "ENST00000434063.3", "ENST00000521028.4",
"ENST00000417262.5", "ENST00000630918.1", "ENST00000524165.7", "ENST00000524964.3", 
"ENST00000609212.1", "ENST00000502515.2", "ENST00000607434.1", "ENST00000534336.4", 
"ENSMUST00000172812.3", "ENST00000501122.3"]

H19 = ["ENST00000414790.10", "ENSMUST00000136359.7"]
Malat1 = ["ENST00000534336.4", "ENSMUST00000172812.3"]
Xist = ["ENST00000429829.6", "ENSMUST00000127786.3"]
Neat1 = ["ENST00000501122.3"]
Airn = ["ENSMUST00000159731.1"]
KCNQ1OT1 = ["ENST00000597346.1"]

# Add a new column to indicate the highlight category
lncRNA_df_pca['Highlight'] = 'Other'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(Cis_Repressive), 'Highlight'] = 'Cis-Repressive'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(Cis_Activating), 'Highlight'] = 'Cis-Activating'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(H19), 'Highlight'] = 'H19'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(Malat1), 'Highlight'] = 'Malat1'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(Xist), 'Highlight'] = 'Xist'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(Neat1), 'Highlight'] = 'Neat1'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(Airn), 'Highlight'] = 'Airn'
lncRNA_df_pca.loc[lncRNA_df_pca.index.isin(KCNQ1OT1), 'Highlight'] = 'KCNQ1OT1'

# Plot "Other" points in grayscale first
other_points = lncRNA_df_pca[lncRNA_df_pca['Highlight'] == 'Other']
grayscale_colors = (other_points['PC4'] - other_points['PC4'].min()) / (other_points['PC4'].max() - other_points['PC4'].min())
fig = px.scatter_3d(
    other_points,
    x='PC1',
    y='PC2',
    z='PC3',
    color=grayscale_colors,
    color_continuous_scale='gray',
    opacity=0.8,
    labels={'color': 'PC4'}
)

# Overlay highlighted points
highlighted_points = lncRNA_df_pca[lncRNA_df_pca['Highlight'] != 'Other']
highlighted_fig = px.scatter_3d(
    highlighted_points,
    x='PC1',
    y='PC2',
    z='PC3',
    color='Highlight',
    hover_name=highlighted_points.index,
    color_discrete_map={
        'Cis-Repressive': 'lightcoral',  # Replaced 'lightred' with 'lightcoral'
        'H19': 'darkred',
        'Xist': 'darkred',
        'Airn': 'red',
        'KCNQ1OT1': 'red',
        'Cis-Activating': 'lightgreen',
        'Malat1': 'darkgreen',
        'Neat1': 'green',
    },
    opacity=0.8
)

# Add highlighted points to the figure
for trace in highlighted_fig.data:
    fig.add_trace(trace)

fig.update_layout(
    legend=dict(
        orientation="h",  # Horizontal orientation
        x=0.5,            # Center the legend horizontally
        xanchor="center", # Anchor the legend at the center
        y=-0.1            # Position the legend below the plot
    ),
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    width=800,
    height=800
)

# Show the plot
fig.show()

In [ ]:
import plotly.io as pio

# Specify the output file path
html_output_path = os.path.join(directory_path, f'PCA_interactive_plot_N{N}_K{K}{"_Corr" if lags else ""}.html')

# Save the Plotly figure as an HTML file
pio.write_html(fig, file=html_output_path, auto_open=False)

print(f"Interactive plot saved to {html_output_path}")